In [49]:
import pickle
import numpy as np
from collections import Counter

In [51]:
entities_names = pickle.load(open("all_indivs.pickle", 'rb'))


with open('all_indivs.txt', 'w') as f:
    for line in entities_names:
        f.write(f"{line}\n")
        
# remove one word entity
# sub any non-alphanum-character w/ ""


entities_alphnum_only = ["".join(char.lower() for char in entity_name if char.isalnum() or char == " ") 
                         for entity_name in entities_names
                        if len(entity_name.split()) > 1]

In [52]:
len(entities_names)

33020

In [8]:
all_edit_dist = np.load("edit_distance_result.npy")

In [32]:
ct = 0
cases = Counter()
for i in range(len(all_edit_dist)):
    
    if (np.sort(all_edit_dist[i])[1] <=1 ):
        query_string = entities_alphnum_only[i].strip()
        query_words = query_string.split()
        
        # for now let's skip names that only composed of one word
        if (len(query_words) == 1):
            continue
        
        match_order = 0
        sorted_arg = np.argsort(all_edit_dist[i])
        sorted_match = np.sort(all_edit_dist[i])
        
#damerauLevenshtein

        ct_col = 0
        dup_ct = -1 # start with -1 because there's at least 1 self match
        while (sorted_match[ct_col] <= 1 ):
            flag = ""
            
            target_string = entities_alphnum_only[sorted_arg[ct_col]].strip()
            proper_val = sorted_match[ct_col]
            ct_col += 1
            
            if (proper_val > 1):
#                 print("throwing out -- ", entities_alphnum_only[sorted_arg[ct_col]],
#                   proper_val, sorted_match[ct_col])
                continue
            
            ## case: exact duplicated
            target_words = target_string.split()
            
            disjoint_words = set(query_words).symmetric_difference(set(target_words))
            
            for spec_keyword in ["et", "and", "y", "amp", "aka"]:
                if spec_keyword in disjoint_words:
                    flag += "find_keyword_"+spec_keyword+";"
                        
            
            if proper_val == 0:
                if len(entities_alphnum_only[i]) == len(target_string):
                    dup_ct += 1
                    if (dup_ct == 0):
                        flag += "self_match;"
                    if (dup_ct > 0):
                        flag += "dup;"
                else:
                    n_add_words = abs(len(entities_alphnum_only[i].split())-len(target_string.split()))
                    if n_add_words == 1:
                        flag += "1_add_word;"
                    else:
                        flag += "add_gt1_words;"
                    
                
                
            
            if proper_val == 1:
                if len(query_string) == len(target_string):
                        flag += "1_char_sub_typo;"                    
                if len(query_string) == len(target_string)+1:
                        flag += "1_char_ins_typo;"
                if len(query_string) == len(target_string)-1:
                        flag += "1_char_del_typo;"
                if abs(len(query_words)-len(target_words)) == 1:
                        flag += "1_typo_1_add_word;"
                if abs(len(query_words)-len(target_words)) > 1:
                        flag += "1_typo_2+_extra_words;"
                    
            
            if (flag == "self_match;" ):
                continue
#             if (flag != ""):
#                 continue
            
            if (flag == ""):
               flag = "??"
                
            match_order += 1
            if (match_order == 1):
                ct += 1
                print("")
                print(ct, i, "len:",len(query_words), "\n","",
                  entities_alphnum_only[i])
            
            print("-",
              target_string,"\t",
                  proper_val, "\t", flag
             )
            cases[flag] += 1


1 7 len: 4 
  baer nancy van norman
- baer nancy van norman 	 0 	 dup;
- nancy bae 	 1 	 1_typo_2+_extra_words;

2 13 len: 2 
  levine mindy
- levine mindy n 	 0 	 1_add_word;

3 26 len: 2 
  simon sj
- simon j dow 	 1 	 1_typo_1_add_word;

4 54 len: 3 
  gates alice a
- alice bates 	 1 	 1_typo_1_add_word;

5 72 len: 3 
  taylor margaret fisk
- taylor margaret 	 0 	 1_add_word;

6 156 len: 3 
  thomas william l
- william e thomas 	 1 	 1_char_sub_typo;

7 167 len: 2 
  ann reinking
- anne reinking 	 1 	 1_char_del_typo;

8 171 len: 2 
  jane pritchard
- pratchard jane 	 1 	 1_char_sub_typo;

9 176 len: 2 
  warren larry
- harry warren 	 1 	 1_char_sub_typo;

10 196 len: 3 
  pohren d e
- pohren de 	 1 	 1_char_ins_typo;1_typo_1_add_word;

11 206 len: 2 
  ernest morgan
- earnest t morgan 	 1 	 1_typo_1_add_word;

12 220 len: 2 
  david todd
- todd davis 	 1 	 1_char_sub_typo;

13 221 len: 2 
  lee sandra
- sondra lee 	 1 	 1_char_sub_typo;

14 242 len: 2 
  sholler ludmilla
- ludmill


134 1446 len: 3 
  white david r
- david white 	 0 	 1_add_word;

135 1450 len: 3 
  kaplan peggy jarrell
- peggy jarrell kaplan 	 0 	 dup;

136 1456 len: 2 
  katz jonathan
- jonathan howard katz 	 0 	 1_add_word;

137 1468 len: 3 
  smith thomas m
- thomas h smith 	 1 	 1_char_sub_typo;

138 1490 len: 2 
  stephan koplowitz
- stephan koplowitz jack freudenheim 	 0 	 add_gt1_words;
- stephen koplowitz 	 1 	 1_char_sub_typo;
- stephan kolowitz 	 1 	 1_char_ins_typo;

139 1501 len: 2 
  morella joe
- joe morello 	 1 	 1_char_sub_typo;

140 1534 len: 3 
  spatt leslie e
- leslie e spatt 	 0 	 dup;

141 1538 len: 3 
  fraser john h
- john fraser 	 0 	 1_add_word;

142 1545 len: 2 
  min tanaka
- mia tanaka 	 1 	 1_char_sub_typo;

143 1546 len: 2 
  the yard
- the school of hard knocks 	 1 	 1_typo_2+_extra_words;

144 1579 len: 2 
  slonimsky juri
- slonimsky yuri 	 1 	 1_char_sub_typo;

145 1597 len: 3 
  erdman joan l
- jean erdman 	 1 	 1_typo_1_add_word;

146 1606 len: 3 
  wilson a 


286 2462 len: 2 
  john hurdle
- john jurdle 	 1 	 1_char_sub_typo;

287 2466 len: 2 
  louis williams
- louie williams 	 1 	 1_char_sub_typo;

288 2468 len: 2 
  betty farrar
- betty farfar 	 1 	 1_char_sub_typo;

289 2475 len: 2 
  letitia ide
- letita ide 	 1 	 1_char_ins_typo;

290 2478 len: 2 
  neal chirico
- neil chirico 	 1 	 1_char_sub_typo;

291 2486 len: 2 
  neil chirico
- neal chirico 	 1 	 1_char_sub_typo;

292 2487 len: 2 
  jose gernando
- jose fernando 	 1 	 1_char_sub_typo;

293 2488 len: 2 
  jose fernando
- jose gernando 	 1 	 1_char_sub_typo;

294 2490 len: 3 
  marta ada thompson
- martha ada thompson 	 1 	 1_char_del_typo;

295 2492 len: 2 
  ella massey
- ella massy 	 1 	 1_char_ins_typo;

296 2494 len: 2 
  lillian pollo
- lillian rollo 	 1 	 1_char_sub_typo;

297 2495 len: 2 
  lilian newcomer
- lillian newcomer 	 1 	 1_char_del_typo;

298 2496 len: 2 
  edward duddley
- edward dudley 	 1 	 1_char_ins_typo;

299 2497 len: 2 
  virginia cartert
- virginia gart


416 3218 len: 2 
  david gold
- david good 	 1 	 1_char_sub_typo;

417 3224 len: 2 
  bette shaler
- betty shaler 	 1 	 1_char_sub_typo;

418 3240 len: 2 
  ray moiler
- ray moller 	 1 	 1_char_sub_typo;

419 3246 len: 2 
  lilian jarvis
- lillian jarvis 	 1 	 1_char_del_typo;

420 3264 len: 2 
  carolyn blazy
- carolyn blazy sheley 	 0 	 1_add_word;

421 3268 len: 3 
  carlos antonio fernandez
- carlos antonio gernandez 	 1 	 1_char_sub_typo;

422 3269 len: 2 
  wisnu wardhana
- wisny wardhana 	 1 	 1_char_sub_typo;

423 3270 len: 3 
  lolita san miguel
- lilita san miguel 	 1 	 1_char_sub_typo;

424 3301 len: 2 
  e hernandez
- mirjam ter linden rafael a hernandez iii 	 1 	 1_typo_2+_extra_words;
- rafael a hernandez iii 	 1 	 1_typo_2+_extra_words;

425 3320 len: 2 
  jose lopez
- jose luis lopez jr 	 0 	 add_gt1_words;

426 3324 len: 3 
  juan antonio hosela
- juan antonio 	 0 	 1_add_word;

427 3329 len: 2 
  maria antonia
- mourinho antonio maria 	 1 	 1_typo_1_add_word;

428 33


545 4353 len: 2 
  miguel lobo
- luís miguel cobo 	 1 	 1_typo_1_add_word;

546 4360 len: 2 
  manfred fischbeck
- manfred frischbeck 	 1 	 1_char_del_typo;

547 4363 len: 2 
  howard sayette
- howard sayatte 	 1 	 1_char_sub_typo;

548 4385 len: 2 
  michael heller
- michael feller 	 1 	 1_char_sub_typo;

549 4392 len: 2 
  paul roberts
- robert benford members of the paul winter consort andrew rudin 	 1 	 1_typo_2+_extra_words;

550 4397 len: 2 
  fredric strobel
- frederic strobel 	 1 	 1_char_del_typo;

551 4402 len: 2 
  armando setina
- armando zetina 	 1 	 1_char_sub_typo;

552 4429 len: 2 
  james boyd
- james wynn jon bond 	 1 	 1_typo_2+_extra_words;

553 4431 len: 2 
  john fisher
- john fischer 	 1 	 1_char_del_typo;
- by john fischer 	 1 	 1_typo_1_add_word;

554 4452 len: 2 
  nancy thueson
- nancy thuesen 	 1 	 1_char_sub_typo;

555 4459 len: 2 
  alexandre blin
- alexandre belin 	 1 	 1_char_del_typo;

556 4483 len: 2 
  elisa monte
- elise monte 	 1 	 1_char_sub_typo;

...


3106 24897 len: 2 
  nikolas gaifullin
- nikolaus gaifullin 	 1 	 1_char_del_typo;

3107 24899 len: 2 
  luis gonzales
- luis gonzalez 	 1 	 1_char_sub_typo;
- luis miguel gonzalez 	 1 	 1_typo_1_add_word;

3108 24932 len: 2 
  branimira ivanova
- branamira ivanova 	 1 	 1_char_sub_typo;

3109 24976 len: 2 
  winter spencer
- earl winter spence 	 1 	 1_typo_1_add_word;

3110 24980 len: 2 
  jennifer aks
- jennifer phillips aka jenny kem 	 1 	 find_keyword_aka;1_typo_2+_extra_words;

3111 25009 len: 2 
  gudrun schretzmeier
- gundrun schretzmeier 	 1 	 1_char_del_typo;

3112 25025 len: 2 
  cristian puig
- christian puig 	 1 	 1_char_del_typo;

3113 25026 len: 2 
  alberto lopez
- alberto c lopez herrera 	 0 	 add_gt1_words;

3114 25030 len: 2 
  george saenz
- george l saenz 	 0 	 1_add_word;

3115 25052 len: 2 
  branamira ivanova
- branimira ivanova 	 1 	 1_char_sub_typo;

3116 25076 len: 2 
  barry phillips
- barry philips 	 1 	 1_char_ins_typo;

3117 25080 len: 2 
  lisa kolosek
-


3204 26571 len: 2 
  pamela z
- belanger pamela j 	 1 	 1_typo_1_add_word;

3205 26585 len: 2 
  harrison burke
- harrison pearse burke 	 0 	 1_add_word;

3206 26609 len: 3 
  leanne iacovetta poirier
- leanne iacovetta 	 0 	 1_add_word;

3207 26620 len: 2 
  laura jacobs
- laura jacob 	 1 	 1_char_ins_typo;

3208 26622 len: 2 
  david hays
- hayo david 	 1 	 1_char_sub_typo;
- david la hay 	 1 	 1_typo_1_add_word;

3209 26639 len: 2 
  marcia sillman
- marcie sillman 	 1 	 1_char_sub_typo;

3210 26641 len: 2 
  marcie sillman
- marcia sillman 	 1 	 1_char_sub_typo;

3211 26642 len: 2 
  mohan khokar
- khokar ashish mohan 	 0 	 1_add_word;

3212 26652 len: 2 
  alex kalman
- maira kalman  alex kalman 	 0 	 add_gt1_words;

3213 26653 len: 4 
  maira kalman  alex kalman
- maira kalman 	 0 	 add_gt1_words;

3214 26673 len: 3 
  nel shelby productions
- nel shelby 	 0 	 1_add_word;

3215 26707 len: 2 
  taylor ciampi
- taylor campi 	 1 	 1_char_ins_typo;

3216 26715 len: 2 
  leonora numm


3290 27473 len: 2 
  jared matthews
- matthew jared perko 	 1 	 1_typo_1_add_word;

3291 27510 len: 2 
  ann kim
- sun yong kim and taik won cho 	 1 	 find_keyword_and;1_typo_2+_extra_words;

3292 27524 len: 2 
  dean rosenthal
- jean rosenthal 	 1 	 1_char_sub_typo;

3293 27526 len: 2 
  rebecca hadley
- rebecca hawley 	 1 	 1_char_sub_typo;

3294 27529 len: 2 
  pauline harris
- pauline kim harris 	 0 	 1_add_word;

3295 27532 len: 2 
  eric durdan
- eric moncell durden jr 	 1 	 1_typo_2+_extra_words;

3296 27534 len: 2 
  brandon harris
- brandyn harris 	 1 	 1_char_sub_typo;

3297 27553 len: 2 
  alice reys
- alice reyes 	 1 	 1_char_del_typo;

3298 27576 len: 2 
  miguel gonzalez
- luis miguel gonzalez 	 0 	 1_add_word;

3299 27612 len: 2 
  ying chien
- chien ming chang 	 1 	 1_typo_1_add_word;

3300 27614 len: 2 
  hannah marcus
- hannah marcus 	 0 	 dup;

3301 27615 len: 2 
  hannah marcus
- hannah marcus 	 0 	 dup;

3302 27639 len: 2 
  whitney browne
- whitney brown 	 1 	 1_


3405 29802 len: 3 
  daryl j hoffman
- darryl j hoffman 	 1 	 1_char_del_typo;

3406 29817 len: 3 
  mary lynn gonsorcik
- may lynn 	 1 	 1_typo_1_add_word;

3407 29838 len: 2 
  b woods
- andrea e woods 	 1 	 1_typo_1_add_word;

3408 29839 len: 2 
  daniel johnson
- daniel johnston 	 1 	 1_char_del_typo;

3409 29866 len: 2 
  youngsil kim
- youngsil kim 	 0 	 dup;

3410 29888 len: 2 
  kim bearsbailey
- kim bearsbailey 	 0 	 dup;

3411 29889 len: 2 
  kim bearsbailey
- kim bearsbailey 	 0 	 dup;

3412 29890 len: 2 
   james lo
- james lo 	 0 	 add_gt1_words;
- james o rubio 	 1 	 1_typo_1_add_word;

3413 29900 len: 3 
  alejandro perez jr
- alejandro perez 	 0 	 1_add_word;

3414 29913 len: 2 
  stephanie powell
- stephanie howell 	 1 	 1_char_sub_typo;

3415 29931 len: 3 
  ousmane omari wiles
- ousmane omari mizrahi wiles 	 0 	 1_add_word;

3416 29935 len: 3 
   eva busta move
- eva busta move 	 0 	 add_gt1_words;
- eva bust a move 	 1 	 1_char_del_typo;1_typo_1_add_word;

3417 299


3518 31685 len: 2 
  robert craft
- robert kraft 	 1 	 1_char_sub_typo;

3519 31695 len: 3 
  patricia de martelaere
- patricia dye 	 1 	 1_typo_1_add_word;

3520 31696 len: 2 
  karen kain
- karen kahn 	 1 	 1_char_sub_typo;

3521 31710 len: 2 
  tyreel simpson
- tyreel simpson 	 0 	 dup;

3522 31711 len: 2 
  tyreel simpson
- tyreel simpson 	 0 	 dup;

3523 31784 len: 3 
  applause theatre book
- applause theatre  cinema books 	 1 	 1_typo_1_add_word;

3524 31804 len: 3 
  johnnie cruise mercer
- johnnie cruise mercer jr 	 0 	 1_add_word;

3525 31812 len: 4 
  johnnie cruise mercer jr
- johnnie cruise mercer 	 0 	 1_add_word;

3526 31848 len: 2 
  alison ozer
- alison myra ozer 	 0 	 1_add_word;

3527 31853 len: 2 
  david gorson
- david gordon 	 1 	 1_char_sub_typo;
- david gordon 	 1 	 1_char_sub_typo;

3528 31856 len: 2 
  katherine miller
- catherine miller 	 1 	 1_char_sub_typo;

3529 31882 len: 2 
  cassandra taylor
- cassandra leigh taylor 	 0 	 1_add_word;

3530 31883 len: 2

In [33]:
cases

Counter({'dup;': 414,
         '1_typo_2+_extra_words;': 73,
         '1_add_word;': 650,
         '1_typo_1_add_word;': 375,
         '1_char_sub_typo;': 1108,
         '1_char_del_typo;': 553,
         '1_char_ins_typo;1_typo_1_add_word;': 4,
         'find_keyword_amp;add_gt1_words;': 49,
         '1_char_ins_typo;': 553,
         'find_keyword_and;add_gt1_words;': 51,
         'add_gt1_words;': 207,
         'find_keyword_and;1_typo_2+_extra_words;': 12,
         '1_char_del_typo;1_typo_1_add_word;': 18,
         'find_keyword_aka;1_typo_2+_extra_words;': 5,
         'find_keyword_amp;1_typo_2+_extra_words;': 17,
         'find_keyword_y;add_gt1_words;': 2,
         'find_keyword_y;1_typo_2+_extra_words;': 2,
         'find_keyword_aka;add_gt1_words;': 6,
         'find_keyword_amp;1_typo_1_add_word;': 2,
         'find_keyword_amp;1_add_word;': 1,
         'find_keyword_y;1_add_word;': 2})

### Examining all of the cases under "duplicated" entities

"duplicated": two entities with matching alphanumeric characters (word order can be permutated).

In [48]:
# find true dup versus special rm dup

ct = 0
ct_n_exact_dup = 0
ct_n_inexact_dup = 0
entities_gt1_word = [e for e in entities_names if len(e.split()) > 1]

for i in range(len(all_edit_dist)):
    
    if (np.sort(all_edit_dist[i])[1] <=1 ):
        query_string = entities_alphnum_only[i].strip()
        query_words = query_string.split()
                
        sorted_arg = np.argsort(all_edit_dist[i])
        sorted_match = np.sort(all_edit_dist[i])

        ct_col = 0
        while (sorted_match[ct_col] <= 0 ):
            
            target_string = entities_alphnum_only[sorted_arg[ct_col]].strip()
            target_words = target_string.split()
            
            ## case: exact duplicated
            ct_col += 1
            if len(entities_alphnum_only[i]) != len(target_string):
                continue
            if sorted_arg[ct_col] == i:
                continue
                
            ref_orig_word = entities_gt1_word[i]
            src_orig_word = entities_gt1_word[sorted_arg[ct_col-1]]
            
            if (ref_orig_word == src_orig_word):
                #exact dup
                ct_n_exact_dup += 1;
                continue
            else:
                print(ref_orig_word, len(ref_orig_word))
                print(src_orig_word, len(src_orig_word))
                ct_n_inexact_dup += 1;
                print("--")
                
print("ct of exact dup:", ct_n_exact_dup)
print("ct of inexact dup:", ct_n_inexact_dup)


Gilvey, John Anthony 20
John Anthony Gilvey 19
--
Noble, Cynthia Nazzaro 22
Cynthia Nazzaro Noble 21
--
McAdams, Dona Ann 17
Dona Ann McAdams 16
--
Forsythe, Ana Marie 19
Ana Marie Forsythe 18
--
Starr, Louis M. 15
Louis M. Starr 14
--
Kaplan, Peggy Jarrell 21
Peggy Jarrell Kaplan 20
--
Spatt, Leslie E. 16
Leslie E Spatt 14
--
Van Norman Baer, Nancy 22
Baer, Nancy Van Norman 22
--
Tsuai, Yung Yung 16
Yung Yung Tsuai 15
--
Greco, Jose 11
Jose Greco 10
--
Louis M. Starr 14
Starr, Louis M. 15
--
Dona Ann McAdams 16
McAdams, Dona Ann 17
--
Raymond Sachse 14
Raymond Sachse. 15
--
Lee Lindsay 11
Lindsay Lee 11
--
Thomas Andrew 13
Andrew Thomas 13
--
Clay Taliaferro 15
Clay Taliaferro. 16
--
Ellen Jacob 11
Jacob, Ellen 12
--
Daniel (Williams) Grossman 26
Daniel Williams Grossman 24
--
Kyle Frederick 14
Kyle Frederick/ 15
--
Shannon Mockli. 15
Shannon Mockli 14
--
Skyler Maxey-Wert 17
Skyler Maxey-Wert. 18
--
Rachel Straus 13
Straus Rachel 13
--
Tara O'Con 10
Tara OCon 9
--
Peter Sloan Lewis 1